# Agenda

1. What is a dictionary?
2. Hash functions
3. Creating a `HashTable` class
4. Retrievals from it
5. Assignments to it
6. Improving the search
7. Magic methods for making it more dict-like
8. Iteration over our dict
9. Modern Python dicts vs. this implementation

# What is a dict?

We in the Python world love dicts. But they're in other languages, too:

- Hashes
- Hash tables
- Hash maps
- Maps
- Key-value stores
- Name-value stores

the idea is that we have pairs of information that we're storing:

- We have the "key,"  we get to determine what it is
    - Keys are unique within a dict
    - They have to be (sort of) immutable
- We have values, which can be anything


In [1]:
d = {'a':10, 'b':20, 'c':30}
d

{'a': 10, 'b': 20, 'c': 30}

In [2]:
len(d)

3

In [3]:
type(d)

dict

In [4]:
d['a'] = 999
d

{'a': 999, 'b': 20, 'c': 30}

In [5]:
d['x']

KeyError: 'x'

In [6]:
# search in a dict for a key
'x' in d

False

In [7]:
'a' in d

True

In [9]:
# how do we remove a key-value pair from our dict?
# dict.pop

d.pop('a')  # removes the key-value pair, and returns the value we removed

KeyError: 'a'

# Rules associated with them

- Keys will be unique. There is no way for a key to repeat itself in a dict
- The keys must be *hashable* , which is sorta kinda the same as immutable.
- Values can be anything at all.

In [10]:
name = 'Reuven'

In [11]:
globals()['name']

'Reuven'

# Hashable and hash functions

If I want to search for an element in a list, how long will it take? It depends.

Dicts are far, far faster than this. The reason is a hash function. The location used by Python to store the key-value pair is based on the key. We run a function, `hash`, on the key, and that gives us a memory location.

If I say

    d['a'] = 100

Python computes `hash('a')`, and then stores ('a', 100) in that location.

In [12]:
hash('a')

-8342762356364650461

In [13]:
hash('b')

-2087163220244094577

In [14]:
hash('c')

3079803896062575534

In [15]:
'a' in d

False

In [16]:
# why doesn't Python let me do this?

mylist = [10, 20, 30]
d[mylist] = 'hello'

TypeError: unhashable type: 'list'

# Exercise: Write a hash function

1. Do not use the builtin function `hash`. Call it `myhash`.
2. It can be really simple!
3. It should work with inputs of type `int`, `float`, `string`, and `tuple`.
4. Given any of these inputs, the function should return an integer.
5. Any other type can raise a `TypeError`.
6. You can use `ord` (which gives the Unicode value for a character) as part of your calculation
7. You should get different values for `hash('abc')` as `hash('cba')`.

You can check the type of a value with `isinstance`, as in `isinstance(10, int)`

In [30]:
def myhash(key):
    if isinstance(key, str):
        total = 0
        for index, one_character in enumerate(key, 1):
            total += ord(one_character) * index
        return total
    elif isinstance(key, int):
        return key
    elif isinstance(key, float):
        return int(key)
    elif isinstance(key, tuple):
        total = 0
        for index, one_item in enumerate(key, 1):
            total += myhash(one_item) * index
        return total
    else:
        raise TypeError(f'Unhashable type: {type(key)}')

In [31]:
myhash('a')

97

In [32]:
myhash('b')

98

In [33]:
myhash('ab')

293

In [34]:
myhash('ba')

292

In [35]:
myhash(10)

10

In [36]:
myhash(123.456)

123

In [38]:
myhash((10, 20, 30))

140

In [39]:
myhash(('hello', 'goodbye'))

7579

# What's the connection between `myhash` and a dict?

- When we assign a key-value pair to our dict, we will use `myhash` to know at what place to put it
- Our dict will start with only 8 locations in which we can put key-value pairs. We'll take the output from `myhash` and use `% 8` on it in order to place the pair.

In [41]:
myhash('abcd') % 8   # remainder after integer division by 8

6

# Exercise: Create a `HashTable` class

1. When you create a new instance of `HashTable`, you'll pass a list of 2-element tuples. The first element will be the key, and the second will be the value.
2. Allocate a list of 8 `None` values in your instance.
3. Iterate over each element in your input list of tuples, hashing the key and putting the key-value pair in the right place.
4. This might end up with some data loss!

Example:

    ht = HashTable([('a',10), ('b',20), ('c',30)])
    print(ht.data)  # we should see a list of 8 elements, all None except 
           # for our 3 pairs

In [44]:
class HashTable:
    def __init__(self, list_of_pairs):
        self.data = [None] * 8

        for key, value in list_of_pairs:
            key_index = myhash(key) % 8
            self.data[key_index] = (key, value)

ht = HashTable([('a', 10), ('b', 20), ('c', 30)])            


In [45]:
ht.data

[None, ('a', 10), ('b', 20), ('c', 30), None, None, None, None]

In [47]:
ht = HashTable([('a', 10), ('b', 20), ('c', 30),
               ('d', 40), ('e', 50), ('f', 60),
               ('g', 70), ('h', 80), ('spite', 100)])            
ht.data


[('h', 80),
 ('a', 10),
 ('b', 20),
 ('c', 30),
 ('d', 40),
 ('e', 50),
 ('f', 60),
 ('spite', 100)]

In [48]:
ht = HashTable([('ab', 10), ('cd', 20), ('ef', 30),
               ('gh', 40), ('ij', 50)])            
ht.data

[None, ('ef', 30), None, ('cd', 20), None, ('ij', 50), None, ('gh', 40)]

In [50]:
myhash('ij') % 8

5

In [51]:
ht.data[5]

('ij', 50)

# Use `[]` to retrieve... or to set

You might have noticed that we use `[]` to retrieve from strings, lists, and tuples, and dicts! We use `[]` to assign to both lists and dicts.

This is thanks to two magic methods.  All of the magic methods start with and end with `__` (double underscore), so they're also known as "dunder methods."

- If we say `a[b]`, then we're really invoking `a.__getitem__(b)`
- If we say `a[b] = c`, then we're effectively invoking `a.__setitem__(b, c)`

If we want to allow people to retrieve from our dict, then we have to implement `__getitem__`. And if we want to allow people to assign to our dict, then we have to implement `__setitem__`.

# Exercise: Magic methods

1. Implement `__repr__`, which returns a string based on our hash table. It should show all of the key-value pairs.
2. Implement `__getitem__`, which will either return the value for that key or raise `KeyError`.
3. Implement `__setitem__`, which will step on any previous key-value pair in that location, with the same hash value.

Again: We are ignoring collisions! If two keys have the same hash value, the last one wins.

In [86]:
class HashTable:
    def __init__(self, list_of_pairs):
        self.data = [None] * 8

        for key, value in list_of_pairs:
            key_index = self.key_location(key)
            self.data[key_index] = (key, value)

    def key_location(self, key):
        return myhash(key) % len(self.data)

    def __repr__(self):
        return str(f'HashTable: {self.data}')

    def __getitem__(self, key):
        key_index = self.key_location(key)
        pair = self.data[key_index]
        if pair[0] == key:
            return self.data[key_index][1]
        else:
            raise KeyError(f'{key} is not in our dict')

    def __setitem__(self, key, value):
        key_index = self.key_location(key)
        self.data[key_index] = (key, value)

ht = HashTable([('a', 10), ('b', 20), ('c', 30)])            

In [87]:
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), None, None, None, None]

In [88]:
ht['i']

KeyError: 'i is not in our dict'

In [89]:
myhash('i') % 8

1

In [90]:
myhash('a') % 8

1

In [91]:
ht['a']

10

In [92]:
ht['abcd'] = 123
ht['cdef'] = 456
ht['ghij'] = 789

ht

HashTable: [None, ('a', 10), ('ghij', 789), ('c', 30), None, None, ('abcd', 123), None]

In [93]:
ht.key_location('cdef')

2

# Two big things we still have to do

1. Avoid collisions
2. Expand our dict as it growso

# Avoiding collisions

The easiest way to do this is to check -- when we are going to assign a key-value pair to a location, we check to see

1. it's empty, with `None`, in which case we can assign to it
2. The key is the same as our current one, in which case we can also assign to it
3. The key exists and is different... in which case we go to the next location and try again

# Exercise: Avoid collisions

1. Modify `key_location` and `__setitem__` such that when you set:
    - If the current location contains `None`, set there as before
    - If the current location contains our key, set there as before
    - If the current location contains a tuple with another key, add 1 to the index and try again
    - Don't forget to wrap around if you're at the end of the list

In [116]:
class HashTable:
    def __init__(self, list_of_pairs):
        self.data = [None] * 8

        for key, value in list_of_pairs:
            key_index = self.key_location(key)
            self.data[key_index] = (key, value)

    def key_location(self, key):
        # get the hash index
        key_index = myhash(key) % len(self.data)

        while True:
            current_item = self.data[key_index]

            # check to see what's in self.data
            # (1) if it's None, then return that index
            if current_item is None:
                return key_index

            # (2) if it contains a tuple, and the key == ours, return index
            current_key, current_value = current_item
    
            if current_key == key:
                return key_index

            # (3) if it contains a tuple, and the key ≠ ours, +1 and try again
            key_index = (key_index + 1) % len(self.data)

    def __repr__(self):
        return str(f'HashTable: {self.data}')

    def __getitem__(self, key):
        key_index = self.key_location(key)
        pair = self.data[key_index]
        if pair is None:
            raise KeyError(f'{key} is not in our dict')
        else:
            return self.data[key_index][1]

    def __setitem__(self, key, value):
        key_index = self.key_location(key)
        self.data[key_index] = (key, value)

ht = HashTable([('a', 10), ('b', 20), ('c', 30)])            

In [117]:
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), None, None, None, None]

In [118]:
ht['i'] = 999

In [119]:
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), ('i', 999), None, None, None]

In [120]:
ht['o'] = 888
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), ('i', 999), None, None, ('o', 888)]

In [121]:
ht['l'] = 777
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), ('i', 999), ('l', 777), None, ('o', 888)]

In [122]:
ht['k'] = 777
ht

HashTable: [None, ('a', 10), ('b', 20), ('c', 30), ('i', 999), ('l', 777), ('k', 777), ('o', 888)]

In [123]:
ht['a'] = 222
ht

HashTable: [None, ('a', 222), ('b', 20), ('c', 30), ('i', 999), ('l', 777), ('k', 777), ('o', 888)]

In [124]:
ht['a']

222

In [125]:
ht['b']

20

In [126]:
ht['i']

999

In [127]:
ht['x']

KeyError: 'x is not in our dict'

# When do we expand our dict?

Python doubles the size of a dict when it becomes 2/3 full.



In [129]:
5/8


0.625